Welcome to the third data of the Regression 5-Day Challenge! Over the last two days of the challenge, we've:

* [Learned about different types of regression (Poisson, linear and logistic) and when to use them](https://www.kaggle.com/rtatman/regression-challenge-day-1)
* [Learned how to fit & evaluate a model with diagnostic plots](https://www.kaggle.com/rtatman/regression-challenge-day-2)

Today, we're going to learn about how to interpret a model and how to tell whether our input variable actually does have a strong relationship to our output variable. To figure this out, we'll need to learn about a new concept: coefficients.

## What's a coefficient?

A coefficient expresses that strength of the relationship between the input value and the output value. You can read a coefficient as "for every increase of the input value by one unit, the output value will change by [whatever number the coefficient is] units". In linear regression, you can think of the coefficient as the slope of the line. Today we're going to fit a model and learn how to see and interpret its coefficients.
___

<center>
[**You can check out a video that goes with this notebook by clicking here.**](https://www.youtube.com/embed/4OnNnu6GqCs)

## Example: Can we predict how likely a hard drive is to fail?
___

Let's see if we can predict the probability that a hard drive will fail based on the Read Error Rate (which is in the column smart_1_normalized in this dataset). Intuitively, I'd expect that a hard drive is more likely to fail if the Read Error Rate is higher, i.e. there are more read errors. 

First we need to set up our environment, though.

In [ ]:
# read in libraries
library(tidyverse)
library(boot)

# data. I'll use the hard_drives dataset & the cameras dataset is for you 
# to use as part of your exerise
hard_drives <- read_csv("../input/hard-drive-test-data/harddrive.csv", n_max = 100000)
cameras <- read_csv("../input/1000-cameras-dataset/camera_dataset.csv")

In [ ]:
#Python Code

import pandas as pd

hard_drives = pd.read_csv('../input/hard-drive-test-data/harddrive.csv', nrows= 90000)
cameras = pd.read_csv("../input/1000-cameras-dataset/camera_dataset.csv")
hard_drives.head()

Because "failure" is a categorical variable with two values, I'm going to use the "binomial" family to fit a logistic regression model. In this dataset 1 indicates that a hard drive did fail and 0 indicates that it didn't.

In [ ]:
# predict probability of failure give the read error rate
model <- glm(failure ~ smart_1_normalized, data = hard_drives, family = "binomial")

In [ ]:
#Python Code
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X = np.array(hard_drives['smart_1_normalized']).reshape(len(hard_drives),1)
y = hard_drives['failure']
logit = LogisticRegression()
logit.fit(X,y)
logit.coef_
logit.intercept_

plt.scatter(X,y)
plt.xlabel('smart_1_normalized')
plt.ylabel('failure')

In [ ]:
# Python Code
import seaborn as sns
sns.lmplot('smart_1_normalized','failure',data=hard_drives,fit_reg=True,hue='failure',scatter_kws={"marker": "D","s": 100})

Great, so now we have a model! But how can we tell what's going on? We can do this using the "summary" function.

> **Note:** I'd recommend doing diagnostic plots at this point, but this is a pretty big model and they took fooooorrrrever to plot, so I'm skipping them here. Don't be lazy like I'm being! :P

In [ ]:
# summary of the model
summary(model)

Wow, that's a lot of output! Let's take it a bit at a time and go through it together. 

**Call**: This is just the call that you made to the function. It will be the exact same code you typed into R. This can be helpful for seeing if you made any typos.

**(Deviance) Residuals:** You can pretty much ignore these for logistic regression. For Poisson or logistic regression, you want these to be more-or-less normally distributed (which is the same thing the top two diagnostic plots are checking). You can check this by seeing if the absolute value of 1Q and 3Q are close(ish) to each other, and if the median is close to 0. The mean is not shown because it's always 0. If any of these are super off then you probably have some weird skew in your data. (This will also show up in your diagnostic plots!)

**Coefficients:** This is the meat of the output.

* **Intercept**: For Poisson and linear regression, this is the predicted output when all our inputs are 0. For logistic regression, this value will be further away from 0 the bigger the difference between the number of observation in each class.. The the standard error represents how uncertain we are about this (lower is better). In this case, because our intercept is far from 0 and our standard error is much smaller than the intercept, we can be pretty sure that one of our classes (failed or didn't fail) has a lot of more observations in it. (In this case it's "didn't fail", thankfully!)
* **Various inputs** (each input will be on a different line): This estimate represents how much we think the output will change each time we increase this input by 1. The bigger the estimate, the bigger the effect of this input variable on the output. The standard error is how certain about it we are. Usually, we can be pretty sure an input is informative is the standard error is 1/10 of the estimate.  So in this case we're pretty sure the intercept is important.
* **Signif. Codes**: This is a key to the significance of each :input and the intercept. These are only correct if you only ever fit one model to your data. (In other words, they’re great for experimental data if you from the start which variables you’re interested in and not as informative for data analysis or variable selection.)

> **Wait, why can't we use statistical significance?** You can, I just wouldn't generally recommend it. In data science you'll often be fitting multiple models using the same dataset to try and pick the best model. If you ever run more than one test for statistical significance on the same dataset, you need to adust your p-value to make up for it. You can think about it this way: if you decide that you'll accept results that are below p = 0.05, you're basically saying that you're ok with being wrong one in twenty times. If you then do five tests, however, and for each one there's a 1/20 chance that you'll be wrong, you now have a 1/4 chance of having been wrong on at least one of those tests... but you don't know which one. You can correct for it ([by multiplying the p-value you'll accept as significant by the number of tests you'll preform](http://mathworld.wolfram.com/BonferroniCorrection.html)) but in practice I find it's generally easier to avoid using p-values altogether. 

**(Dispersion parameter for binomial family taken to be 1):** You'll only see this for Poisson and binomial (logistic) regression. It's just letting you know that there has been an additional scaling parameter added to help fit the model. You can ignore it. :)

**Null deviance:** The null deviance tells us how well we can predict our output *only* using the intercept. Smaller is better.

**Residual deviance:** The residual deviance tells us how well we can predict our output using the intercept and our inputs. Smaller is better. The bigger the difference between the null deviance and residual deviance is, the more helpful our input variables were for predicting the output variable.

**AIC:** The AIC is the "Akaike information criterion" and it's an estimate of how well your model is describing the patterns in your data. It's mainly used for comparing models trained on the same dataset. If you need to pick between models, the model with the *lower* AIC is doing a better job describing the variance in the data.

**Number of Fisher Scoring iterations:** This is just a measure of how long it took to fit you model. You can safely ignore it. 

OK, that was a lot! Let's get the summary of our model again and quickly go over what this tells us.

In [ ]:
# summary of the model
summary(model)

From looking at the Deviance Residuals, we can tell that there's a really big skew in our residuals. Since the mean of the residuals is always 0 (due to the way that the model is fit) the median is lower than the mean. For that matter, the 3rd quartile is lower than the mean! However, since this is logistic regression and we don't need our residuals to be normally distributed, this doesn't really matter.

From looking at the coefficients, we can tell that our classes are pretty unbalanced (because our intercept is far from 0 and our standard error is much smaller than it). We can also tell that our input, smart_1_normalized, doesn't have a very strong relationship to our output because the standard error (0.009) is more than 1/10 of our estimate (0.02). Our suspicions are confirmed when we look at the difference between the null & residual deviance: adding the smart_1_normalized term barely improved our deviance at all! 

So, based on this model, we can say that it seems like smart_1_normalized, on its own, isn't a really good predictor of whether a hard drive is going to fail or not. We can double check this intution by plotting our model.

In [ ]:
ggplot(hard_drives, aes(x = smart_1_normalized, y = failure)) + # draw a 
    geom_point() + # add points
    geom_smooth(method = "glm", # plot a regression...
    method.args = list(family = "binomial")) # ...from the binomial family

This plot does confirm what we've learned from our model: our classes are very imbalanced and the input variable is not that helpful. It looks like this model is pretty much always just predicting that a hard drive *won't* fail, since most hard drives don't.

## Your turn!
___

Now it's your turn to come up with a model and interpret it!

1. Pick a question to answer to using the Cameras dataset. Pick a variable to predict and one variable to use to predict it.
2. Fit a GLM model of the appropriate family. (Check out [Monday's challenge](https://www.kaggle.com/rtatman/regression-challenge-day-1) if you need a refresher).
3. *Optional but recommended:* Plot diagnostic plots for your model. Does it seem like your model is a good fit for your data? If you're fitting a linear or Poisson model, are the residuals normally distributed (no patterns in the first plot and the points in the second plot are all in a line)? Are there any influential outliers?
4. Check out your model using the summary() function. Does your input variable have a strong relationship to the output variable you're predicting?
5. Write a couple sentences describing what you've learned from your model. (It could just be that it's not a very good model!)
5. Plot your two variables & use "geom_smooth" and the appropriate family to fit and plot a model. Does this confirm what you learned from examining your model?
6. *Optional:* If you want to share your analysis with friends or to ask for help, you’ll need to make it public so that other people can see it.
    * Publish your kernel by hitting the big blue “publish” button. (This may take a second.)
    * Change the visibility to “public” by clicking on the blue “Make Public” text (right above the “Fork Notebook” button).
    * Tag your notebook with 5daychallenge

In [ ]:
# your work goes here! :)
# Python Code for working with Cameras Dataset
cameras.columns
cameras.describe()

In [ ]:
# Python Code
# Find the outliers in the camera prices i.e. any values lying within 3 std dev.
cam_price_outliers = cameras[~(np.abs(cameras.Price-cameras.Price.mean())<=(3*cameras.Price.std()))]
cam_price_outliers.head()


In [ ]:
cameras['Release date'].unique()

In [ ]:
log = LogisticRegression()
x = np.array(cameras['Max resolution']).reshape(len(cameras),1)
y = cameras['Price']
log.fit(x, y)

sns.lmplot('Max resolution','Price',data=cameras,fit_reg=True)



Want more? Ready for a different dataset? [This notebook](https://www.kaggle.com/rtatman/datasets-for-regression-analysis/) has additional dataset suggestions for you to practice regression with. 